In [18]:
from qiskit import QuantumCircuit
import numpy as np

In [19]:
def dj_function(num_qubits, type, number):
    """
    Create a random Deutsch-Jozsa function. num_qubits is number of qubits, type is 'balanced' or 'constant,' and number is output value for constant function.
    """

    qc = QuantumCircuit(num_qubits + 1) # Create quantum circuit with one additional quantum register for minus state.

    if (num_qubits > 15 or num_qubits < 0):
        raise Exception("Input Error: num_qubits should be between 0 and 15, inclusive") # Handling Exception: num_qubits

    if (type == "constant"):
        if number != 0 and number != 1: raise Exception("Input Error: Please specify number as either 0 or 1") # Handling Exception: number
        # If constant function value is given as 1
        if (number == 1):
            qc.x(num_qubits)
        return qc
    
    elif (type == "balanced"):
        # Chooses half the possible input states to convert
        on_states = np.random.choice(
            range(2**num_qubits),  # numbers to sample from
            2**num_qubits // 2,  # number of samples
            replace=False,  # makes sure states are only sampled once
        )
        
        # Function to add x gates to bitstring bits that need to be flipped.
        def add_x(qc, bit_string):
            for qubit, bit in enumerate(reversed(bit_string)):
                if bit == "1":
                    qc.x(qubit)
            return qc
        
        # For half the states chosen in on_states, adds the xor operation.
        for state in on_states:
            # print(state)
            qc.barrier()  # Barriers are added to help visualize how the functions are created. They can safely be removed.
            qc = add_x(qc, f"{state:0b}") # Flips bits for xor consideration.
            qc.mcx(list(range(num_qubits)), num_qubits) # Applies xor.
            qc = add_x(qc, f"{state:0b}") # Reverses bit flip for next round of xor consideration.
        qc.barrier()

        return qc
    
    else: raise Exception("Input Error: Please specify type as either 'constant' or 'balanced'") # Handling Exception: type

In [20]:
display(dj_function(2, 'balanced', 1).draw())

░ ┌───┐     ┌───┐ ░                 ░ 
q_0: ─░─┤ X ├──■──┤ X ├─░────────■────────░─
      ░ ├───┤  │  ├───┤ ░ ┌───┐  │  ┌───┐ ░ 
q_1: ─░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░─
      ░ └───┘┌─┴─┐└───┘ ░ └───┘┌─┴─┐└───┘ ░ 
q_2: ─░──────┤ X ├──────░──────┤ X ├──────░─
      ░      └───┘      ░      └───┘      ░

In [21]:
def compile_circuit(function: QuantumCircuit):
    """
    Compiles a circuit for use in the Deutsch-Jozsa algorithm.
    """
    n = function.num_qubits - 1
    qc = QuantumCircuit(n + 1, n)
    qc.x(n)
    qc.h(n)
    qc.barrier()
    qc.h(range(n))
    qc.compose(function, inplace=True)
    qc.h(range(n))
    qc.measure(range(n), range(n))
    display(qc.draw())
    return qc

In [22]:
# Code to run algorithm
from qiskit_aer import AerSimulator
def dj_algorithm(function: QuantumCircuit):
    """
    Determine if a Deutsch-Jozsa function is constant or balanced.
    """
    qc = compile_circuit(function)

    result = AerSimulator().run(qc, shots=1, memory=True).result()
    measurements = result.get_memory()
    if "1" in measurements[0]:
        return "balanced"
    return "constant"

In [24]:
# Defined Function
f = dj_function(4, "balanced", 0)
# Determine using Algorithm
display(dj_algorithm(f))

░ ┌───┐ ░ ┌───┐     ┌───┐ ░       ░                 ░ ┌───┐»
q_0: ───────────░─┤ H ├─░─┤ X ├──■──┤ X ├─░───■───░────────■────────░─┤ X ├»
                ░ ├───┤ ░ ├───┤  │  ├───┤ ░   │   ░        │        ░ └───┘»
q_1: ───────────░─┤ H ├─░─┤ X ├──■──┤ X ├─░───■───░────────■────────░──────»
                ░ ├───┤ ░ └───┘  │  └───┘ ░   │   ░        │        ░ ┌───┐»
q_2: ───────────░─┤ H ├─░────────■────────░───■───░────────■────────░─┤ X ├»
                ░ ├───┤ ░ ┌───┐  │  ┌───┐ ░   │   ░ ┌───┐  │  ┌───┐ ░ ├───┤»
q_3: ───────────░─┤ H ├─░─┤ X ├──■──┤ X ├─░───■───░─┤ X ├──■──┤ X ├─░─┤ X ├»
     ┌───┐┌───┐ ░ └───┘ ░ └───┘┌─┴─┐└───┘ ░ ┌─┴─┐ ░ └───┘┌─┴─┐└───┘ ░ └───┘»
q_4: ┤ X ├┤ H ├─░───────░──────┤ X ├──────░─┤ X ├─░──────┤ X ├──────░──────»
     └───┘└───┘ ░       ░      └───┘      ░ └───┘ ░      └───┘      ░      »
c: 4/══════════════════════════════════════════════════════════════════════»
                                                                           »
«          ┌───┐ ░                 ░ ┌───┐     ┌───┐ ░                 ░ ┌───┐»
«q_0: ──■──┤ X ├─░────────■────────░─┤ X ├──■──┤ X ├─░────────■────────░─┤ X ├»
«       │  └───┘ ░ ┌───┐  │  ┌───┐ ░ └───┘  │  └───┘ ░        │        ░ ├───┤»
«q_1: ──■────────░─┤ X ├──■──┤ X ├─░────────■────────░────────■────────░─┤ X ├»
«       │  ┌───┐ ░ └───┘  │  └───┘ ░ ┌───┐  │  ┌───┐ ░ ┌───┐  │  ┌───┐ ░ └───┘»
«q_2: ──■──┤ X ├─░────────■────────░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░──────»
«       │  ├───┤ ░ ┌───┐  │  ┌───┐ ░ └───┘  │  └───┘ ░ ├───┤  │  ├───┤ ░      »
«q_3: ──■──┤ X ├─░─┤ X ├──■──┤ X ├─░────────■────────░─┤ X ├──■──┤ X ├─░──────»
«     ┌─┴─┐└───┘ ░ └───┘┌─┴─┐└───┘ ░      ┌─┴─┐      ░ └───┘┌─┴─┐└───┘ ░      »
«q_4: ┤ X ├──────░──────┤ X ├──────░──────┤ X ├──────░──────┤ X ├──────░──────»
«     └───┘      ░      └───┘      ░      └───┘      ░      └───┘      ░      »
«c: 4/════════════════════════════════════════════════════════════════════════»
«                                                                             »
«          ┌───┐ ░ ┌───┐┌─┐         
«q_0: ──■──┤ X ├─░─┤ H ├┤M├─────────
«       │  ├───┤ ░ ├───┤└╥┘┌─┐      
«q_1: ──■──┤ X ├─░─┤ H ├─╫─┤M├──────
«       │  └───┘ ░ ├───┤ ║ └╥┘┌─┐   
«q_2: ──■────────░─┤ H ├─╫──╫─┤M├───
«       │        ░ ├───┤ ║  ║ └╥┘┌─┐
«q_3: ──■────────░─┤ H ├─╫──╫──╫─┤M├
«     ┌─┴─┐      ░ └───┘ ║  ║  ║ └╥┘
«q_4: ┤ X ├──────░───────╫──╫──╫──╫─
«     └───┘      ░       ║  ║  ║  ║ 
«c: 4/═══════════════════╩══╩══╩══╩═
«                        0  1  2  3

'balanced'